THings to manifest

- All intact lowland rainorest is preserved   (use Original_LU2_1600.shp)     DONE


- Special management polygons are created     (use Special_mangmet_ares_wetland.shp)  Done 
   - MMI, 
   - buffer around wetlands
   - Tafuna Rainforest area
    - aasu forest preserve

- Coastal development reduced and a natural buffer of mangrove forest is created around coastal areas at a 6 ft rise level   Done 
   - do the existing mangrove buffer, but also add in one that pereserves coastal forest or rock araes
    
- increase and consolidataion of agroforest areas   (do with  their location in the expansion model )

- do the stream flood area thing too make it bigger    (Using a 5 m buffer on either side of stream)    Done


In [25]:
# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))

import numpy as np
import arcpy
import os
from arcpy.sa import *
import pandas as pd
#import gdal
from arcpy import env

arcpy.env.overwriteOutput = True # make sure overwrite files is on

cel_size = 5     # in m   Note that a cell size any smaller than 5 is going to push the files over the 100 mb limit on Github
# projection definition 
sr_project = arcpy.SpatialReference(32702)   # Project dataset into WGS84

# LU Dictionary for later

Lukeys = {400:'Cultivated_Land', 
500:'Agroforest', 
700:'Open_Space', 
800:'Developed_Woodlands', 
1000:'Buildings', 
1000:'Impervious_Surfaces', 
1200:'Mangroves', 
1300:'Exposed_Rock', 
1400:'Grassland', 
1500:'Upland_Scrub', 
1600:'Lowland_Rainforest', 
1700:'Successional_Scrub', 
2100:'Montane_Rainforest', 
-999:"no Data",0:"no Data"}

In [26]:
GIS_FOLDER = os.path.join('..', 'Raw_GIS_Data')

# path to the grid bound
Grid_shp = os.path.join(GIS_FOLDER, 'grid_bound.shp')

if not os.path.exists(os.path.join('.', 'output//')):
    os.makedirs(os.path.join('.', 'output//'))
                        
if not os.path.exists(os.path.join('.', 'output//Figures//')):
    os.makedirs(os.path.join('.', 'output//Figures//'))
fig_path =  (os.path.join('.', 'output//Figures//'))

if not os.path.exists(os.path.join('.', "WorkSpaces")):
    os.makedirs(os.path.join('.', "WorkSpaces"))
workspace =  os.path.join('.', "WorkSpaces")

# define where the land is too steep to build
steep_land = os.path.join(GIS_FOLDER, 'Slopes', 'steep_land_25deg.shp')

# define where the original low rainforest is to protect 

Forest_preserve_land = os.path.join(GIS_FOLDER, 'LU_Scenarios', 'Original_LU2_1600.shp')

In [29]:
""" This is the function that selects a land use, then bufferes it, then adds it back to the map. It will be repeated below for multiple nested land uses

    maptoclip is the working version of the land use map, on iteration 1 its the original map, then its the ones that have the buffered areas added in incrementally
    orig_map_to_buffer is the original unadulterated map
    LU_type is the land use getting worked on each iteration
    buffer_dist_m isthe distance of buffering to change, main parameter to modify
"""

def clip_da_use(maptoclip, orig_map_to_buffer, LU_type, buffer_dist_m, steep_land, Forest_preserve_land,  iteration_num):
    
    tempselect = os.path.join(workspace, "temp.shp") 
    tempbuffer = os.path.join(workspace, "temp_buffer.shp")
    buffer_clipped_slopes = os.path.join(workspace, "temp_buffer_clipped.shp")
    buffer_clipped_slopes_forest = os.path.join(workspace, "temp_buffer_clipped_forest.shp")
    clipped_Map = os.path.join(workspace, "LU_shp_erased.shp")
    output_file = os.path.join(workspace, "intermediate_LU_new_{}.shp".format(iteration_num))
    
    # Buffer the features of interest
    arcpy.Select_analysis(orig_map_to_buffer, tempselect, "LBLCLASS = '{}'".format(LU_type))     # select just the LU type desired save as tempselect.shp
    arcpy.Buffer_analysis(tempselect, tempbuffer, buffer_dist_m, dissolve_option="NONE")         # Make a tenporary buffer file of that land use bit 
    arcpy.RepairGeometry_management(tempbuffer)                                                  # fix broken things
    
    # erase out areas where there are large slopes
    arcpy.Erase_analysis(tempbuffer, steep_land,  buffer_clipped_slopes)     
    
    # erase out areas where we are protecting forest
    arcpy.Erase_analysis(buffer_clipped_slopes, Forest_preserve_land,  buffer_clipped_slopes_forest)           
    
    # erase out the new bufferd land area zones
    arcpy.Erase_analysis(maptoclip, buffer_clipped_slopes_forest, clipped_Map)    
    
    # add the new land use type back to the big LU map 
    arcpy.Merge_management([buffer_clipped_slopes_forest, clipped_Map], output_file)

In [62]:
# hyperparameters
Agroforest_buffer = 10
Cultivated_Land_buffer = 7
Developed_Woodlands_buffer = 2
Open_Space_buffer = 3.5
Impervious_Surfaces_buffer = 4
Buildings_buffer = 5

LU_list =         [ 'Cultivated_Land',  'Agroforest',   'Developed_Woodlands',      'Open_Space',      'Impervious_Surfaces',      'Buildings']
bufferdist_list = [ Cultivated_Land_buffer, Agroforest_buffer,   Developed_Woodlands_buffer, Open_Space_buffer, Impervious_Surfaces_buffer, Buildings_buffer]

In [63]:
# Buffer each land use to expand it respective to the hyperparameters in the cells above

for shed in range(0,33):                          # for each of the watershed clips
    print("Working on shed No. {}..................................................".format(shed))
    
    # Reset the starting map to the new watershed 
    arcpy.CopyFeatures_management(os.path.join(GIS_FOLDER, "Land_use", "Split_into_watersheds", 'Shed_fragments', 'LU_simp3_clipped_{}.shp'.format(shed)), os.path.join(workspace, "intermediate_LU_new_0.shp"))

    Original_Map = os.path.join(workspace, 'intermediate_LU_new_0.shp')    # this is just for shorter syntax on calling the original map

    for iteration_num in range(1,7):        # for each of the land use classes we want to buffer on  (references the lists above)
        maptoclip          = os.path.join(workspace, "intermediate_LU_new_{}.shp".format(iteration_num-1))  # this is where you put in the map from the last iteration
        orig_map_to_buffer = Original_Map
        LU_type            = LU_list[iteration_num-1]
        buffer_dist_m      = bufferdist_list[iteration_num-1]
        print("now buffering {}".format(LU_type))

        clip_da_use(maptoclip, orig_map_to_buffer, LU_type, buffer_dist_m, steep_land, Forest_preserve_land, iteration_num)   # run the clip buffer erase merge function

    # trim off any excess that has spilled out of the watershed
    arcpy.Clip_analysis(os.path.join(workspace, "intermediate_LU_new_6.shp"), Original_Map, os.path.join(workspace, "Finished_LU_{}.shp".format(shed)))

# Merge all the litte buffered watershed ones together
# Create 99, the final merged set
arcpy.CopyFeatures_management(os.path.join(workspace, 'Finished_LU_0.shp'), os.path.join(workspace, "Finished_LU_99.shp"))
for p in range(1,33):
    arcpy.Append_management(os.path.join(workspace, 'Finished_LU_{}.shp'.format(p)), os.path.join(workspace, "Finished_LU_99.shp"))  #agregate function

Working on shed No. 0..................................................
now buffering Cultivated_Land
now buffering Agroforest
now buffering Developed_Woodlands
now buffering Open_Space
now buffering Impervious_Surfaces
now buffering Buildings
Working on shed No. 1..................................................
now buffering Cultivated_Land
now buffering Agroforest
now buffering Developed_Woodlands
now buffering Open_Space
now buffering Impervious_Surfaces
now buffering Buildings
Working on shed No. 2..................................................
now buffering Cultivated_Land
now buffering Agroforest
now buffering Developed_Woodlands
now buffering Open_Space
now buffering Impervious_Surfaces
now buffering Buildings
Working on shed No. 3..................................................
now buffering Cultivated_Land
now buffering Agroforest
now buffering Developed_Woodlands
now buffering Open_Space
now buffering Impervious_Surfaces
now buffering Buildings
Working on shed No. 4...

##  Here use  take the SMAS and turn them into forest or wetland based on what they are coded as 

In [64]:
# clip out the SMAs
arcpy.Erase_analysis(os.path.join(workspace, "Finished_LU_99.shp"), os.path.join(GIS_FOLDER, "LU_Scenarios", "Special_mangmet_ares_wetland.shp"), os.path.join(workspace, "tempSMAspuka.shp"))

# add the SMA areas back to the big LU map 
arcpy.Merge_management([os.path.join(workspace, "tempSMAspuka.shp"),os.path.join(GIS_FOLDER, "LU_Scenarios", "Special_mangmet_ares_wetland.shp")], os.path.join(workspace, "Finished_LU_99.shp"))

<Result '.\\WorkSpaces\\Finished_LU_99.shp'>

## Effects of stream flooding to open space

In [65]:
### This block takes anything impervious from within 5 m of a stream and turns it to open space

# select the impervious and buildings areas from the orignal LU map make a temp shapefile
arcpy.Select_analysis(os.path.join(GIS_FOLDER, 'Land_use',  'LU_Big_simp_3.shp'), 
                      os.path.join(workspace, "temp205.shp"), '"LBLCLASS" = \'Buildings\' OR "LBLCLASS" = \'Impervious_Surfaces\'')

# find  where these buildings/imp surfaces intersect the stream buffer 
arcpy.Intersect_analysis([os.path.join(workspace, "temp205.shp"), os.path.join(GIS_FOLDER,'Streams', 'Streams_5mBuffer.shp') ], os.path.join(workspace, "temp_Bldg2flood205.shp"), "ALL")

# change the Loclass to open space
with arcpy.da.UpdateCursor(os.path.join(workspace, "temp_Bldg2flood205.shp"), "LBLCLASS") as cursor:
    for row in cursor:
        row[0] = "Open_Space"
        cursor.updateRow(row)
        
# change the Lucode to open space
with arcpy.da.UpdateCursor(os.path.join(workspace, "temp_Bldg2flood205.shp"), "LU2") as cursor:
    for row in cursor:
        row[0] = 700
        cursor.updateRow(row)

# erase areas where the new open space will be        
arcpy.Erase_analysis(os.path.join(workspace, "Finished_LU_99.shp"), os.path.join(workspace, "temp_Bldg2flood205.shp"), os.path.join(workspace, "temp205_puka.shp"))

# merge the new open space areas back into the big map
arcpy.Merge_management([os.path.join(workspace, "temp205_puka.shp"), os.path.join(workspace, "temp_Bldg2flood205.shp")], os.path.join(workspace, "Finished_LU_99.shp"))


<Result '.\\WorkSpaces\\Finished_LU_99.shp'>

## Effects of sea level rise on coasts 

In [66]:
# add in efects of sea level rise based on 6 ft scenario (see references folder in github)   (from https://coast.noaa.gov/slrdata/)
# in short this finds ares affected by 6ft SLR then turns them from other land uses to mangroves except if they were originally coastal bare rock, in which case it turns them back to bare rock

  
# THis just creates a shapefile of wherer there is unconsolidated shore and removes out the upland exposed rock featuers 
# buffer the coastline by 250 m
arcpy.Buffer_analysis(os.path.join(GIS_FOLDER, "SLR", "Present_day_coastline.shp"), os.path.join(workspace, "temp_buffer2.shp"), 250, dissolve_option="ALL") 
# select the rocky areas from the orignal LU map
arcpy.Select_analysis(os.path.join(GIS_FOLDER, 'Land_use', 'LU_Big_simp_3.shp'), os.path.join(workspace, "temp2.shp"), '"LBLCLASS" = \'Exposed_Rock\'')
# get only the rock coastal areas 
arcpy.Intersect_analysis([os.path.join(workspace, "temp2.shp"), os.path.join(workspace, "temp_buffer2.shp")], os.path.join(workspace, "temp_Coastal_ROck.shp"), "ALL")

# Now take the final result from the big LU buffering process and clip out  ALLL areas where SLR will impact and turn them into mangroves

# "AS_Tutuila_SLR_3ft_man_Dissolve.shp"   is simply an intersect of the  'LU_Big_simp_3.shp' and the 6 ft SLR polygons. Then the LU2 is changed to a mangrove code manually. 
arcpy.Erase_analysis(os.path.join(workspace, "Finished_LU_99.shp"), os.path.join(GIS_FOLDER, 'SLR', "AS_Tutuila_SLR_6ft_man_Dissolve.shp"), os.path.join(workspace, "tempSLR.shp"))
# add the new mangrove areas back to the big LU map 
arcpy.Merge_management([os.path.join(workspace, "tempSLR.shp"), os.path.join(GIS_FOLDER, 'SLR', "AS_Tutuila_SLR_6ft_man_Dissolve.shp")], os.path.join(workspace, "Finished_LU_99.shp"))

# Finally clip out the new mangrove areas that were on top of coastal rock and make them coasta rock once again.
arcpy.Erase_analysis(os.path.join(workspace, "Finished_LU_99.shp"), os.path.join(workspace, "temp_Coastal_ROck.shp"), os.path.join(workspace, "tempRock2wholeI.shp"))
# add the new mangrove areas back to the big LU map 
arcpy.Merge_management([os.path.join(workspace, "tempRock2wholeI.shp"), os.path.join(workspace, "temp_Coastal_ROck.shp")], os.path.join(workspace, "Finished_LU_99.shp"))

<Result '.\\WorkSpaces\\Finished_LU_99.shp'>

In [78]:
# Ultimately this should be run on the re-merged together shapefile of all watersheds

# This converts the original and the new coverage to rasters to run the percentage calculation stats

# just give a link to the shapefile of major watersheds
Origlandusefile = os.path.join(GIS_FOLDER, 'Land_use',  'LU_Big_simp_3.shp')
#  Merge in the grid boundary into the finished shapefile to create accurate grid coverage 
arcpy.Erase_analysis(Grid_shp, os.path.join(workspace, "Finished_LU_99.shp"),  os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'LU_shp_bound.shp'), os.path.join(workspace, "Finished_LU_99.shp")], os.path.join(workspace, 'LU_shp_ready.shp'))

# convert to raster.asc
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "LU2", os.path.join(workspace, "LU_raster"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "LU_raster"), os.path.join(workspace, "new_LU_grid.asc"))

# this deos the same with the original one for comparison
#  Merge in the grid bound into the shapefile to create accurate grid coverage 
arcpy.Erase_analysis(Grid_shp, Origlandusefile,  os.path.join(workspace, 'LU_shp_bound.shp'))
arcpy.Merge_management([os.path.join(workspace, 'LU_shp_bound.shp'), Origlandusefile], os.path.join(workspace, 'LU_shp_ready.shp'))

# convert to raster.asc
arcpy.PolygonToRaster_conversion(os.path.join(workspace, 'LU_shp_ready.shp'), "LU2", os.path.join(workspace, "LU_raster"), cell_assignment="MAXIMUM_AREA",  cellsize=cel_size)
arcpy.RasterToASCII_conversion(os.path.join(workspace, "LU_raster"), os.path.join(workspace, "original_LU_grid.asc"))

arr = arcpy.RasterToNumPyArray(os.path.join(workspace, "original_LU_grid.asc"),  nodata_to_value=-999)
unique, counts = np.unique(arr, return_counts=True)
arr_stats_original = np.asarray((unique, counts)).T

arr = arcpy.RasterToNumPyArray(os.path.join(workspace, "new_LU_grid.asc"),  nodata_to_value=-999)
unique, counts = np.unique(arr, return_counts=True)
arr_stats_new = np.asarray((unique, counts)).T

# this takes the cell counts for each land use type in the new and old rasters and compares them percentage-wise
new_dic = {}; old_dic = {}
for i in arr_stats_new: 
    new_dic[i[0]] = i[1]
for i in arr_stats_original: 
    old_dic[i[0]] = i[1]
    
for i in old_dic:
    try:     
        oabs = old_dic[i]
        nabs = new_dic[i]
        pctchange = ((nabs/oabs)-1)*100
        oldarea = oabs*25*0.000001
        newarea = nabs*25*0.000001
        print("LU type_{:.0f}, {}, is {:.2f}% different from {:.2f} km2 to {:.2f} km2".format(i, Lukeys[i], pctchange, oldarea, newarea))
    except:
        print("LU {} does not exist in one coverage".format(i))

LU -999 does not exist in one coverage
LU type_0, no Data, is 0.02% different from 563.68 km2 to 563.80 km2
LU type_400, Cultivated_Land, is -52.82% different from 10.97 km2 to 5.18 km2
LU type_500, Agroforest, is 52.49% different from 8.17 km2 to 12.46 km2
LU type_700, Open_Space, is -20.17% different from 6.87 km2 to 5.49 km2
LU type_800, Developed_Woodlands, is -64.72% different from 5.26 km2 to 1.85 km2
LU type_1000, Impervious_Surfaces, is 64.08% different from 8.33 km2 to 13.67 km2
LU type_1200, Mangroves, is 891.31% different from 0.35 km2 to 3.49 km2
LU type_1300, Exposed_Rock, is -29.33% different from 2.13 km2 to 1.50 km2
LU type_1400, Grassland, is -34.06% different from 1.79 km2 to 1.18 km2
LU type_1500, Upland_Scrub, is -6.93% different from 1.41 km2 to 1.32 km2
LU type_1600, Lowland_Rainforest, is 0.37% different from 85.86 km2 to 86.18 km2
LU type_1700, Successional_Scrub, is -26.59% different from 4.83 km2 to 3.55 km2
LU type_2100, Montane_Rainforest, is -1.29% differen

In [76]:
(sum(old_dic.values())-22547157)*25


136321075

In [75]:
old_dic

{-999: 1,
 0: 22547157,
 400: 438986,
 500: 326775,
 700: 274950,
 800: 210244,
 1000: 333346,
 1200: 14064,
 1300: 85041,
 1400: 71461,
 1500: 56553,
 1600: 3434404,
 1700: 193289,
 2100: 13729}

In [36]:
# IMPORTANT! 
### When pushing to github uncoment and run this cell, this file is over 100 mb

    
# delete the Aggregated shapefile to push to GitHub (>100 mb) uncheck this to get the shapefile back

#arcpy.Delete_management(os.path.join(workspace, "Finished_LU_99.shp"))

In [12]:
Scn3 Log 

# hyperparameters
Agroforest_buffer = 14
Cultivated_Land_buffer = 4
Developed_Woodlands_buffer = 2
Open_Space_buffer = 4
Impervious_Surfaces_buffer = 4
Buildings_buffer = 5

LU -999 does not exist in one coverage
LU type_0, no Data, is 0.02% different
LU type_400, Cultivated_Land, is -60.65% different
LU type_500, Agroforest, is 62.76% different
LU type_700, Open_Space, is -16.67% different
LU type_800, Developed_Woodlands, is -67.13% different
LU type_1000, Impervious_Surfaces, is 64.02% different
LU type_1200, Mangroves, is 891.28% different
LU type_1300, Exposed_Rock, is -29.50% different
LU type_1400, Grassland, is -35.48% different
LU type_1500, Upland_Scrub, is -7.34% different
LU type_1600, Lowland_Rainforest, is 0.36% different
LU type_1700, Successional_Scrub, is -27.44% different
LU type_2100, Montane_Rainforest, is -1.30% different




Agroforest_buffer = 20
Cultivated_Land_buffer = 4
Developed_Woodlands_buffer = 2
Open_Space_buffer = 4
Impervious_Surfaces_buffer = 4
Buildings_buffer = 5

LU -999 does not exist in one coverage
LU type_0, no Data, is 0.02% different
LU type_400, Cultivated_Land, is -67.24% different
LU type_500, Agroforest, is 74.17% different
LU type_700, Open_Space, is -16.68% different
LU type_800, Developed_Woodlands, is -67.14% different
LU type_1000, Impervious_Surfaces, is 63.99% different
LU type_1200, Mangroves, is 891.29% different
LU type_1300, Exposed_Rock, is -29.84% different
LU type_1400, Grassland, is -39.09% different
LU type_1500, Upland_Scrub, is -7.89% different
LU type_1600, Lowland_Rainforest, is 0.34% different
LU type_1700, Successional_Scrub, is -29.63% different
LU type_2100, Montane_Rainforest, is -1.30% different


# hyperparameters
Agroforest_buffer = 10
Cultivated_Land_buffer = 10
Developed_Woodlands_buffer = 2
Open_Space_buffer = 2
Impervious_Surfaces_buffer = 6
Buildings_buffer = 6

LU -999 does not exist in one coverage
LU type_0, no Data, is 0.02% different
LU type_400, Cultivated_Land, is -3.91% different
LU type_500, Agroforest, is -8.11% different
LU type_700, Open_Space, is -42.63% different
LU type_800, Developed_Woodlands, is -64.49% different
LU type_1000, Impervious_Surfaces, is 83.88% different
LU type_1200, Mangroves, is 890.96% different
LU type_1300, Exposed_Rock, is -30.49% different
LU type_1400, Grassland, is -43.11% different
LU type_1500, Upland_Scrub, is -8.98% different
LU type_1600, Lowland_Rainforest, is 0.30% different
LU type_1700, Successional_Scrub, is -31.88% different
LU type_2100, Montane_Rainforest, is -1.31% different


# hyperparameters
Agroforest_buffer = 15
Cultivated_Land_buffer = 3
Developed_Woodlands_buffer = 2
Open_Space_buffer = 2
Impervious_Surfaces_buffer = 5
Buildings_buffer = 5


LU -999 does not exist in one coverage
LU type_0, no Data, is 0.02% different
LU type_400, Cultivated_Land, is 3.07% different
LU type_500, Agroforest, is -19.44% different
LU type_700, Open_Space, is -35.62% different
LU type_800, Developed_Woodlands, is -59.28% different
LU type_1000, Impervious_Surfaces, is 71.20% different
LU type_1200, Mangroves, is 891.16% different
LU type_1300, Exposed_Rock, is -29.32% different
LU type_1400, Grassland, is -35.33% different
LU type_1500, Upland_Scrub, is -7.37% different
LU type_1600, Lowland_Rainforest, is 0.35% different
LU type_1700, Successional_Scrub, is -27.18% different
LU type_2100, Montane_Rainforest, is -1.30% different


In [ ]:
# Scenario 1
Agroforest_buffer = 50
Cultivated_Land_buffer = 3
Developed_Woodlands_buffer = 5
Open_Space_buffer = 5
Impervious_Surfaces_buffer = 6 
Buildings_buffer = 5

LU -999 does not exist in one coverage
LU type_0, no Data, is 0.14% different
LU type_400, Cultivated_Land, is 7.12% different
LU type_500, Agroforest, is 39.27% different
LU type_700, Open_Space, is -12.84% different
LU type_800, Developed_Woodlands, is -62.74% different
LU type_1000, Impervious_Surfaces, is 96.47% different
LU type_1200, Mangroves, is 287.43% different
LU type_1300, Exposed_Rock, is -60.66% different
LU type_1400, Grassland, is -45.35% different
LU type_1500, Upland_Scrub, is -9.57% different
LU type_1600, Lowland_Rainforest, is -6.76% different
LU type_1700, Successional_Scrub, is -33.11% different
LU type_2100, Montane_Rainforest, is -0.63% different

In [ ]:
LOG scen3



